In [ ]:
import pygame
import numpy as np
import random
import sys

In [ ]:
def game_map_maker(map_size,random_cells=0,pulsars=0,tubs=0,gliders=0,blinkers=0):

    map_array = np.zeros((map_size,map_size))
    print(map_array)

    for cell in range(random_cells):
        map_array[random.randint(0,map_size-1)][random.randint(0,map_size-1)] = 1

    for _ in range(tubs):
        x = random.randint(1, map_size - 2)
        y = random.randint(1, map_size - 2)

        map_array[x - 1][y] = 1
        map_array[x + 1][y] = 1
        map_array[x][y - 1] = 1
        map_array[x][y + 1] = 1

    for _ in range(blinkers):
        x = random.randint(1, map_size - 2)
        y = random.randint(1, map_size - 2)
        map_array[x][y:y + 3] = 1
        
    for i in range(pulsars):
      x_offset = random.randint(0, map_size - 18)
      y_offset = random.randint(0, map_size - 18)

      pulsar_pattern = np.zeros((17, 17))
      pulsar_pattern[2, 4:7] = 1
      pulsar_pattern[4:7, 7] = 1
      pulsar_pattern += pulsar_pattern.T
      pulsar_pattern += pulsar_pattern[:, ::-1]
      pulsar_pattern += pulsar_pattern[::-1, :]

      map_array[x_offset:x_offset+17, y_offset:y_offset+17] = pulsar_pattern  # Adjusted to 17

    for i in range(gliders):
        x_offset = random.randint(0, map_size - 8)
        y_offset = random.randint(0, map_size - 8)
    
        glider = [[1, 0, 0],
          [0, 1, 1],
          [1, 1, 0]]
        u = np.zeros((8, 8))
        u[:3, :3] = glider

        rotation_degrees = random.choice([0, 90, 180, 270])
        rotated_u = np.rot90(u, rotation_degrees // 90)

        map_array[x_offset:x_offset+8, y_offset:y_offset+8] = rotated_u



    return map_array

In [ ]:
def game_of_life(map_array):
    alive_coords = np.transpose(np.nonzero(map_array))
    new_map_array = np.zeros_like(map_array)
    
    map_size = map_array.shape[0]
    
    neighbor_sum = np.zeros((map_size, map_size), dtype=int)
    
    for x, y in alive_coords:
        x_min = max(x - 1, 0)
        x_max = min(x + 1, map_size - 1)
        y_min = max(y - 1, 0)
        y_max = min(y + 1, map_size - 1)
        neighbor_sum[x_min:x_max + 1, y_min:y_max + 1] += 1
    
    # Apply the game of life rules using vectorized operations
    new_map_array[(map_array == 1) & ((neighbor_sum < 3) | (neighbor_sum > 4))] = 0
    new_map_array[(map_array == 1) & ((neighbor_sum == 3) | (neighbor_sum == 4))] = 1
    new_map_array[(map_array == 0) & (neighbor_sum == 3)] = 1

    live_cells = np.count_nonzero(new_map_array)
    
    return new_map_array, live_cells



In [ ]:
map_array = game_map_maker(map_size=1000,random_cells=0,tubs=0,pulsars=100,blinkers=100,gliders=50)
map_size = map_array.shape[0]
scale = 2

display_size = (800, 800)
pygame.init()
pygame.display.set_caption("ArcBros Game of Life")
screen = pygame.display.set_mode(display_size)
screen.fill((0,0,0))
live_cells = np.count_nonzero(map_array)

font = pygame.font.SysFont('Arial', 25)
frame_text = font.render("", True, (255,255,255))

screen = pygame.display.set_mode(display_size)    #set the display mode, window title and FPS clock
pygame.display.set_caption('screen')
FPSCLOCK = pygame.time.Clock()
    
fps = 10
time=0
speed=10*scale

visible=True
camera_x=0
camera_y=0


while True:
    keys = pygame.key.get_pressed()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            sys.exit()

    screen.fill(0)

    # Camera movement logic
    if keys[pygame.K_RIGHT]:
        camera_x += speed
    if keys[pygame.K_LEFT]:
        camera_x -= speed
    if keys[pygame.K_DOWN]:
        camera_y += speed
    if keys[pygame.K_UP]:
        camera_y -= speed

    # Draw alive cells within the visible grid
    visible_min_x = max(camera_x // scale, 0)
    visible_max_x = min((camera_x + display_size[0]) // scale, map_size)
    visible_min_y = max(camera_y // scale, 0)
    visible_max_y = min((camera_y + display_size[1]) // scale, map_size)

    

    for x in range(visible_min_x, visible_max_x):
        for y in range(visible_min_y, visible_max_y):
            if map_array[x, y] == 1:
                pygame.draw.rect(screen, (255, 0, 0), ((x * scale - camera_x), (y * scale - camera_y), scale, scale), 0)

    frame_text = font.render(f"Frames: {str(time)} alive cells: {live_cells} ",  True, (255,255,255))
    screen.blit(frame_text, (10, 10))

    pygame.display.update()
    pygame.display.flip()

    map_array, live_cells = game_of_life(map_array)
    time += 1

